In [ ]:
# probly Tutorial — Dropconnect Transformation 

This notebook is meant as a, practical introduction to the **Dropconnect transformation** in `probly`.
The goal is not to be mathematically perfect, but to give you an intuition.

We will slowly build up from the very basic idea of *normal* Dropout to the slightly more advanced idea of
a **Dropout transformation that makes a model uncertainty‑aware**. After that, we look at a small PyTorch
example and inspect how the transformation changes the model.

# Introduction to Dropconnect and the Dropconnect Transformation


## 1. Concept: What is Dropconnect (normal) vs Dropconnect Transformation?


## 2. Quickstart (PyTorch)

Below: build a small MLP, apply `dropconnect(model, p)`, and inspect the modified architecture.

## 3. Uncertainty via DropConnect

